<a href="https://colab.research.google.com/github/simonabisiani/chatgpt_employment/blob/main/ChatGPT_employment_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT and employment project
*Authors: Simona Bisiani, Al-Hussein Abutaleb*

The purpose of this notebook is to perform and evaluate a text analysis of tweets relating to ChatGPT and employment. The project sits within research conducted at the Surrey Institute for Artificial Intelligence by Dr Eddy Zhu and Dr Erin Ling. 

The notebook includes the following sections:
- data collection (this is not to be re-run, it's only included for transparency purposes)
- summary statistics and characteristics of the data
- topic modeling
- sentiment analysis

The necessary data for processing code in this notebook can be found at this [GitHub repository](https://github.com/simonabisiani/chatgpt_employment). We will use the code below to load the data in the notebook.

In [2]:
# LOADING THE DATA
!git clone https://github.com/simonabisiani/chatgpt_employment.git

Cloning into 'chatgpt_employment'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 41 (delta 7), reused 24 (delta 2), pack-reused 0
Unpacking objects: 100% (41/41), 6.11 MiB | 2.98 MiB/s, done.
Filtering content: 100% (2/2), 141.96 MiB | 40.72 MiB/s, done.


In [ ]:
!pip install rpy2==3.5.1   # needed to run R code in colab; restart runtime after running this chunk

In [1]:
%load_ext rpy2.ipython

# this command allows us to run both R and Python code throughout the notebook. 
#Whenever we need to run R code, we simply need to include %%R at the beginning of the code chunk

## Data collection
Upon recommendations from Dr Ling, we queried Twitter for tweets relating to search **chatgpt** *AND* any of the following words:
**"employment", "employability", "employer","employee", "unemployment",      "unemployed", "job", "work", "skill", "taking over", "replace human", "performance", "concern", "insecurity", "fear", "threat", "opportunities", "training","creation", "displacement", "occupation", "earning","future demand",
"education", "labour", "labor", "task", "industry", "workforce", "shortage","collaboration", "collaborate", "human ai team", "team", "economic", "economy"** 

On 24th February we collected, using the Twitter API and the R package *academictwitteR*, any tweets containing our query **between 30th November 2022 and 24th February 2023**. The data collection was only possible through an Academic Twitter API developer account, due to: the number of tweets wanted, and the possibility to search the full archive. With a standard developer account (e.g. Elevated), this would have not been possible.

The code we wrote to collect the data is this:

```
chatgpt_full_collection <- data.frame()
and <- "chatgpt"
or <- c("employment",
        "employability",
        "employer",
        "employee",         
        "unemployment",
        "unemployed",
        "job", 
        "work",
        "skill",
        "taking over",
        "replace human",
        "performance",
        "concern",
        "insecurity",
        "fear",
        "threat",
        "opportunities",
        "training",
        "creation", 
        "displacement", 
        "occupation",
        "earning", 
        "future demand",
        "education",
        "labour",
        "labor", 
        "task", 
        "industry",
        "workforce",
        "shortage", 
        "collaboration", 
        "collaborate", 
        "human ai team",
        "team",
        "economic", 
        "economy")

queries <- paste(and, or)

for (i in queries) {
  iteration_i <- get_all_tweets(
    query = i,
    start_tweets = "2022-11-30T00:00:00Z",
    end_tweets = "2023-02-24T00:00:00Z",
    n = Inf,
    bearer_token = bearer_token, # NOTE! the bearer token is not included in this notebook, making this code chunk impossible to run unless in possession of a Twitter API Academic account
    data_path = "chatgpt_full_collection/")
  iteration_i$query <- i  
  chatgpt_full_collection <- bind_rows(chatgpt_full_collection, iteration_i)}

saveRDS(chatgpt_full_collection, "chatgpt_full_collection.RDS")
data <- readRDS("chatgpt_full_collection.RDS")
tweets <- data %>% select(text, query, created_at, author_id)

file <- toJSON(tweets)
write(file, "full_dataset_chatgpt.json")
```



There is little knowledge on query formulation on Twitter. Our **assumption** was **that the singular form of the word** (or its canonical form) **would return** tweets containing **the plural form too**. However, using another function from the academictwitteR package, which allows us to check for the number of tweets resulting from a query without needing to download the data, we found out **this appears not to be true**.

We ran the code below to obtain the number of tweets for each of the singular/plural pairs, and saved the data. Let's load the data in the notebook so we can visualise the differences in counts between singular/plural queries.

```
and <- "chatgpt"
or <- c("employee",
        "employees", # post collection insertion
        "job", 
        "jobs", # post collection insertion
        "skill",
        "skills", # post collection insertion
        "concern",
        "concerns", # post collection insertion
        "threat",
        "threats", # post collection insertion
        "occupation", # captures occupations
        "occupations", # post collection insertion
        "earning", # captures earnings
        "earnings", # post collection insertion
        "task", # captures tasks
        "tasks", # post collection insertion
        "team", # NEW SUGGESTION (would also capture the above)
        "teams") # post collection insertion

queries <- paste(and, or)

query_counts <- data.frame()
for (i in queries) {
  iteration_i <- count_all_tweets(i,
                                  "2022-11-30T00:00:00Z",
                                  "2023-02-24T00:00:00Z",
                                  bearer_token,
                                  n = 500)
  iteration_i$query <- i  # Create a new column "query" and assign it the value of i
  query_counts <- bind_rows(query_counts, iteration_i)}

saveRDS(query_counts, "query_counts.RDS")
```

Let's now see what we can figure out from comparing singular/plural versions of the same word.

In [1]:
%%R 
query_counts <- read_csv("/content/chatgpt_employment/query_counts.csv")

library(RColorBrewer)

# JOB VS JOBS
query_counts %>% 
  filter(str_detect(query, "job")) %>% 
ggplot(aes(x = as.Date(start), y = tweet_count, fill = query)) +
  geom_bar(stat = "identity", alpha = 0.8, position = "dodge") +
  scale_fill_brewer(palette = "Set1", name = "Query")+
  theme_minimal()+
  xlab("")+
  ylab("tweet count") +
  ggtitle("")+
  labs(subtitle = "")

# SKILL VS SKILLS
query_counts %>% 
  filter(str_detect(query, "skill")) %>% 
  ggplot(aes(x = as.Date(start), y = tweet_count, fill = query)) +
  geom_bar(stat = "identity", alpha = 0.8, position = "dodge") +
  scale_fill_brewer(palette = "Set1", name = "Query")+
  theme_minimal()+
  xlab("")+
  ylab("tweet count") +
  ggtitle("")+
  labs(subtitle = "")

# THREAT VS THREATS
query_counts %>% 
  filter(str_detect(query, "threat")) %>% 
  ggplot(aes(x = as.Date(start), y = tweet_count, fill = query)) +
  geom_bar(stat = "identity", alpha = 0.8, position = "dodge") +
  scale_fill_brewer(palette = "Set1", name = "Query")+
  theme_minimal()+
  xlab("")+
  ylab("tweet count") +
  ggtitle("")+
  labs(subtitle = "")

# TASK VS TASKS
query_counts %>% 
  filter(str_detect(query, "task")) %>% 
  ggplot(aes(x = as.Date(start), y = tweet_count, fill = query)) +
  geom_bar(stat = "identity", alpha = 0.8, position = "dodge") +
  scale_fill_brewer(palette = "Set1", name = "Query")+
  theme_minimal()+
  xlab("")+
  ylab("tweet count") +
  ggtitle("")+
  labs(subtitle = "")

# EMPLOYEE VS EMPLOYEES
query_counts %>% 
  filter(str_detect(query, "employee")) %>% 
  ggplot(aes(x = as.Date(start), y = tweet_count, fill = query)) +
  geom_bar(stat = "identity", alpha = 0.8, position = "dodge") +
  scale_fill_brewer(palette = "Set1", name = "Query")+
  theme_minimal()+
  xlab("")+
  ylab("tweet count") +
  ggtitle("")+
  labs(subtitle = "")

UsageError: Cell magic `%%R` not found.


Following the analysis above, we decided to run a second data collection for the plural form of the words.

```
library(tidyverse)
and <- "chatgpt"
or <- c("employees", # post collection insertion
        "jobs", # post collection insertion
        "skills", # post collection insertion
        "concerns", # post collection insertion
        "threats", # post collection insertion
        "occupations", # post collection insertion
        "earnings", # post collection insertion
        "tasks", # post collection insertion
        "collaborations") # post collection insertion

queries <- paste(and, or)

chatgpt_second_collection <- data.frame()
for (i in queries) {
  iteration_i <- get_all_tweets(
    query = i,
    start_tweets = "2022-11-30T00:00:00Z",
    end_tweets = "2023-02-24T00:00:00Z",
    n = Inf,
    bearer_token = bearer_token,
    data_path = "chatgpt_second_collection/")
  iteration_i$query <- i  # Create a new column "query" and assign it the value of i
  chatgpt_second_collection <- bind_rows(chatgpt_second_collection, iteration_i)}

saveRDS(chatgpt_second_collection, "chatgpt_second_collection.RDS")
```



## Summary statistics and characteristics of the data

Essentially, 

In [ ]:
%%R

# keep only unique observations
filtered <- tweets %>%
  group_by(text) %>%
  slice(1)

# mark retweets and identity author
filtered <- filtered %>%
  mutate(rt = str_detect(full_text, "RT.*:\\s"),
         author = str_extract(full_text, "RT.*:\\s"))

# how many tweets from the same person have been retweeted?
filtered %>% ungroup() %>% select(author_id) %>% drop_na() %>% count(author_id, sort = TRUE) %>% 
  filter(n > 50) %>% 
  ggplot(aes(x = reorder(author_id, n), y = n)) +
  geom_bar(stat = "identity") +
  xlab("") +
  ylab("Number of retweets")+
  ggtitle("Each bar is a tweet, its height the number of times it has been retweeted")+
  theme(axis.text.x = element_blank())+
  theme_minimal()

# doing the same for chatgpt full dataset
tweets <- tweets %>%
  mutate(rt = str_detect(full_text, "RT.*:\\s"),
         author = str_extract(full_text, "RT.*:\\s"))

# visualising
tweets %>% ungroup() %>% select(author) %>% drop_na() %>% count(author, sort = TRUE) %>% slice_head(n = 10) %>%
  ggplot(aes(x = reorder(author, n), y = n)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  ggtitle("Top 10")

filtered <- filtered %>%
  mutate(text_clean = str_remove(full_text, "RT.*@"))

filtered %>% count(text_clean) # remove http links